# Setup

In [4]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.30.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.mavasbel.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.30.2"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.mavasbel.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [5]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("CQLENG_ALLOW_SCHEMA_MANAGEMENT", "True")

'True'

# Session creation

In [6]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_nodes = [
    DefaultEndPoint(CASSANDRA_NODE_HOSTNAMES[j], CASSANDRA_NODE_RPC_PORTS[j])
    for j in range(CASSANDRA_TOTAL_NODES)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)
print(
    f"JDBC URL: jdbc:cassandra://{','.join([f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes])}"
)

auth_provider = PlainTextAuthProvider(
    username=CASSANDRA_INIT_USER, password=CASSANDRA_INIT_PASSWORD
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

🔗 Connecting to: ['cassandra-node-1.mavasbel.vpn.itam.mx:9041', 'cassandra-node-2.mavasbel.vpn.itam.mx:9042', 'cassandra-node-3.mavasbel.vpn.itam.mx:9043']
JDBC URL: jdbc:cassandra://cassandra-node-1.mavasbel.vpn.itam.mx:9041,cassandra-node-2.mavasbel.vpn.itam.mx:9042,cassandra-node-3.mavasbel.vpn.itam.mx:9043
✅ Connected to cluster: cassandra-cluster.mavasbel.vpn.itam.mx
🌐 Nodes found: 3


In [7]:
import pprint
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.options.display.html.use_mathjax = True
pd.options.display.html.border = 1
pd.options.display.html.table_schema = False
pd.options.display.expand_frame_repr = True

display(
    pd.DataFrame(
        [cassandra_host.__dict__ for cassandra_host in cluster.metadata.all_hosts()]
    )
    .sort_index()
    .transpose()
)

,0,1,2
endpoint,cassandra-node-3.mavasbel.vpn.itam.mx:9043,10.15.30.2:9041,10.15.30.2:9042
conviction_policy,<cassandra.policies.SimpleConvictionPolicy object at 0x00000240528CE330>,<cassandra.policies.SimpleConvictionPolicy object at 0x000002405084ACF0>,<cassandra.policies.SimpleConvictionPolicy object at 0x000002405084B350>
host_id,33ab3e51-8f84-43e8-9170-4f92e26443bc,fdc42583-1f24-44ab-a3db-f9e1f960be91,82155aa9-3aea-4ad0-92fa-3831d5c268b0
_datacenter,dc1,dc1,dc1
_rack,rack1,rack1,rack1
lock,<unlocked _thread.RLock object owner=0 count=0 at 0x00000240546A74C0>,<unlocked _thread.RLock object owner=0 count=0 at 0x00000240529014C0>,<unlocked _thread.RLock object owner=0 count=0 at 0x00000240506A0A80>
is_up,True,True,True
listen_address,172.19.0.4,NaN,NaN
listen_port,7003.0,NaN,NaN
broadcast_address,10.15.30.2,10.15.30.2,10.15.30.2


In [8]:
# from cassandra.cluster import ResultSet

# peers_v2: ResultSet = session.execute("SELECT * FROM system.peers_v2")
# locals: ResultSet = session.execute("SELECT * FROM system.local")

# for peer_v2 in peers_v2:
#     print(" | ".join([f"{col}: {peer_v2[col]}" for col in peers_v2.column_names]))
# print("=" * 100)
# for local in locals:
#     print(" | ".join([f"{col}: {local[col]}" for col in locals.column_names]))
# print("=" * 100)

### Keyspace

In [9]:
keyspace_name = "generic_analytics"
if CASSANDRA_START_FROM_SCRATCH:
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_name}")

In [10]:
keyspace_name = "generic_analytics"
session.execute(
    f"""
CREATE KEYSPACE IF NOT EXISTS {keyspace_name} 
WITH replication = {{
    'class': 'NetworkTopologyStrategy', 
    'dc1': {CASSANDRA_TOTAL_NODES}
}}
"""
)
session.set_keyspace(keyspace_name)

# Create Table
# Partition Key: city (distributes data)
# Clustering Column: user_id (sorts data within city)
session.execute(
    """
CREATE TABLE IF NOT EXISTS user_metrics (
    city text,
    user_id uuid,
    username text,
    session_duration int,
    last_access timestamp,
    PRIMARY KEY (city, user_id)
);
"""
)

### Insert

In [11]:
from faker import Faker

fake = Faker()

In [12]:
import uuid
from datetime import datetime
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

# 1. Prepare your statement outside the loop
query = """
INSERT INTO user_metrics (city, user_id, username, session_duration, last_access)
VALUES (?, ?, ?, ?, ?)
"""
prepared = session.prepare(query)
# prepared.consistency_level = ConsistencyLevel.QUORUM
# statement = SimpleStatement(query, consistency_level=ConsistencyLevel.QUORUM)

# 2. Create the Batch object
# LOGGED ensures atomicity but adds disk overhead, UNLOGGED is faster
batch_records = 200
batch = BatchStatement(batch_type=BatchType.LOGGED)
batch.consistency_level = ConsistencyLevel.QUORUM
print(f"Preparing batch of {batch_records} records...")
for _ in range(batch_records):
    batch.add(
        prepared,
        (
            fake.city(),
            uuid.uuid4(),
            fake.user_name(),
            fake.random_int(min=1, max=3600),
            datetime.now(),
        ),
    )

# 3. Execute the entire batch at once
session.execute(batch)
print("Batch successfully committed to the cluster.")

Preparing batch of 200 records...
Batch successfully committed to the cluster.


### Query

In [13]:
from cassandra.cluster import ResultSet
from typing import cast

print(cast(ResultSet, session.execute("SELECT count(*) FROM user_metrics")).one())

{'count': 200}


In [14]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT * FROM user_metrics LIMIT 100")
for row in rows.current_rows:
    print(row)

{'city': 'Lake Jaredborough', 'user_id': UUID('4609566e-28ca-4cb4-8c5f-92cfbf4162fc'), 'last_access': datetime.datetime(2026, 1, 26, 23, 1, 53, 170000), 'session_duration': 3574, 'username': 'lauralowe'}
{'city': 'Elizabethburgh', 'user_id': UUID('dfafd2bc-d4da-4b92-88a7-70371f43d3d2'), 'last_access': datetime.datetime(2026, 1, 26, 23, 1, 53, 173000), 'session_duration': 1312, 'username': 'gaymeghan'}
{'city': 'Robertotown', 'user_id': UUID('a7f24ccf-cff7-411d-858b-b2369bf3e01d'), 'last_access': datetime.datetime(2026, 1, 26, 23, 1, 53, 174000), 'session_duration': 3275, 'username': 'gprice'}
{'city': 'Morrischester', 'user_id': UUID('f82dff70-dfe4-4b17-9837-88083bcdf84f'), 'last_access': datetime.datetime(2026, 1, 26, 23, 1, 53, 159000), 'session_duration': 744, 'username': 'matthewcopeland'}
{'city': 'Hannahburgh', 'user_id': UUID('ee2e6d3f-ef10-4e9c-af76-fdc6c1594710'), 'last_access': datetime.datetime(2026, 1, 26, 23, 1, 53, 161000), 'session_duration': 2948, 'username': 'rhodesjam

In [15]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT DISTINCT city FROM user_metrics", [])
for row in rows.current_rows:
    print(row)

{'city': 'Lake Jaredborough'}
{'city': 'Elizabethburgh'}
{'city': 'Robertotown'}
{'city': 'Morrischester'}
{'city': 'Hannahburgh'}
{'city': 'Lake Melissaside'}
{'city': 'Angelaberg'}
{'city': 'East Timothy'}
{'city': 'Brownport'}
{'city': 'South Bobby'}
{'city': 'Lake Martha'}
{'city': 'Jeremiahshire'}
{'city': 'South Joanberg'}
{'city': 'Lisamouth'}
{'city': 'Savagemouth'}
{'city': 'Jamestown'}
{'city': 'Salazarstad'}
{'city': 'Laurenton'}
{'city': 'Russellview'}
{'city': 'Frenchburgh'}
{'city': 'Sandovalburgh'}
{'city': 'East Johnton'}
{'city': 'Port Kylefort'}
{'city': 'North Tammy'}
{'city': 'Stewartport'}
{'city': 'Matthewbury'}
{'city': 'West Bobby'}
{'city': 'Gutierrezside'}
{'city': 'Blackstad'}
{'city': 'Simpsonmouth'}
{'city': 'Lake David'}
{'city': 'New Samuel'}
{'city': 'Fisherburgh'}
{'city': 'Shaffertown'}
{'city': 'Sotofurt'}
{'city': 'Valdezfort'}
{'city': 'Port Jerry'}
{'city': 'East Hayley'}
{'city': 'Katelynhaven'}
{'city': 'Annechester'}
{'city': 'Chantown'}
{'city'

### Find nodes storing random data

In [16]:
import random
import pprint

random_token = random.randint(-9223372036854775808, 9223372036854775807)
query = "SELECT * FROM user_metrics WHERE token(city) >= %s LIMIT 1"
row: dict = cast(ResultSet, session.execute(query, [random_token])).one()
print(f"Random user_metric: {pprint.pformat(row)}")

prepared = session.prepare("SELECT * FROM user_metrics WHERE city=? AND user_id=?")
bound = prepared.bind([row['city'], row['user_id']])
routing_key = bound.routing_key
nodes = cluster.metadata.get_replicas(keyspace_name, routing_key)

print(f"Nodes storing '{row['city']}':")
for node in nodes:
    print(f" - Host: {node.address}, Gossip Port: {node.broadcast_port}")

Random user_metric: {'city': 'Meganborough',
 'last_access': datetime.datetime(2026, 1, 26, 23, 1, 53, 164000),
 'session_duration': 3596,
 'user_id': UUID('450929d5-07b5-4473-89f3-782718e98058'),
 'username': 'wandacolon'}
Nodes storing 'Meganborough':
 - Host: cassandra-node-3.mavasbel.vpn.itam.mx, Gossip Port: 7003
 - Host: 10.15.30.2, Gossip Port: 7002
 - Host: 10.15.30.2, Gossip Port: 7001


### ORM-like

In [17]:
from typing import cast
from cassandra.cqlengine import columns
from cassandra.cqlengine.query import ModelQuerySet
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table, create_keyspace_network_topology
from cassandra.cqlengine import connection

# 1. Connect the engine
connection.set_session(session)


# 2. Define your "Generic" Model
class UserMetrics(Model):
    __table_name__ = "user_metrics"

    # FIRST primary_key=True is the Partition Key
    # city = columns.Text(primary_key=True)
    city = columns.Text(primary_key=True, partition_key=True)

    # SECOND primary_key=True is the Clustering Key
    user_id = columns.UUID(primary_key=True, default=uuid.uuid4)

    # Attributes (Data)
    username = columns.Text(index=True)
    session_duration = columns.Integer()
    last_access = columns.DateTime()


# 3. Create Keyspace and Sync Table (Equivalent to CREATE TABLE)
create_keyspace_network_topology(keyspace_name, {"dc1": CASSANDRA_TOTAL_NODES})
sync_table(UserMetrics)

In [18]:
# 4. Use it like an ORM
# Create UserMetrics
new_metric: UserMetrics = UserMetrics.create(
    city=fake.city(),
    username=fake.name(),
    session_duration=120,
    last_access=datetime.now(),
)
new_metric.save()
print(f"Saved user_metris: {new_metric}")


# 5. Query UserMetrics
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering() # Non-recommended, used for non-primary key queries
)
for user_metric in user_metrics:
    print(f"Query user_metris: {user_metric}")

Saved user_metris: UserMetrics <city=Hansenshire, user_id=5dd4f717-1f45-44f4-ac8d-501961b23270>
Query user_metris: UserMetrics <city=Lake Jaredborough, user_id=4609566e-28ca-4cb4-8c5f-92cfbf4162fc>
Query user_metris: UserMetrics <city=Elizabethburgh, user_id=dfafd2bc-d4da-4b92-88a7-70371f43d3d2>
Query user_metris: UserMetrics <city=Robertotown, user_id=a7f24ccf-cff7-411d-858b-b2369bf3e01d>
Query user_metris: UserMetrics <city=Morrischester, user_id=f82dff70-dfe4-4b17-9837-88083bcdf84f>
Query user_metris: UserMetrics <city=Hannahburgh, user_id=ee2e6d3f-ef10-4e9c-af76-fdc6c1594710>
Query user_metris: UserMetrics <city=Lake Melissaside, user_id=bea23348-feac-444d-9b36-8333ba7aed13>
Query user_metris: UserMetrics <city=Angelaberg, user_id=95f7dd94-ad5c-40a8-b7ad-438d5906c17f>
Query user_metris: UserMetrics <city=East Timothy, user_id=879a163f-3be0-4fcc-b0c5-c84ff1800fe1>
Query user_metris: UserMetrics <city=Brownport, user_id=d3260b9f-5dfe-40b1-8064-d053f26f51c6>
Query user_metris: UserMet

### Insert/Create

In [19]:
from typing import cast

cast(ModelQuerySet, UserMetrics.ttl(86400)).create(
    city=fake.city(),
    username=fake.user_name(),
    session_duration=fake.random_int(10, 1000),
    last_access=fake.date_time(),
)

UserMetrics(city='Port Stephanie', user_id=UUID('45b38be6-73cb-492c-aa92-9e6976fc7d47'), username='hernandezrobin', session_duration=463, last_access=datetime.datetime(1986, 11, 3, 17, 27))

In [20]:
from typing import cast
from cassandra.cqlengine.query import BatchQuery
from cassandra.cqlengine.query import ModelQuerySet

with BatchQuery() as b:
    for _ in range(100):
        cast(ModelQuerySet, UserMetrics.batch(b)).create(
            city=fake.city(),
            username=fake.user_name(),
            session_duration=fake.random_int(10, 1000),
            last_access=fake.date_time(),
        )

### Delete

In [21]:
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering() # Non-recommended, used for non-primary key queries
)
for user_metric in user_metrics:
    user_metric.delete()
    print(f"Delete user_metris: {user_metric}")

Delete user_metris: UserMetrics <city=Lake Jaredborough, user_id=4609566e-28ca-4cb4-8c5f-92cfbf4162fc>
Delete user_metris: UserMetrics <city=Elizabethburgh, user_id=dfafd2bc-d4da-4b92-88a7-70371f43d3d2>
Delete user_metris: UserMetrics <city=Port Alicia, user_id=5f53643e-b890-41e3-a5b6-2050f1d70e78>
Delete user_metris: UserMetrics <city=Robertotown, user_id=a7f24ccf-cff7-411d-858b-b2369bf3e01d>
Delete user_metris: UserMetrics <city=Morrischester, user_id=f82dff70-dfe4-4b17-9837-88083bcdf84f>
Delete user_metris: UserMetrics <city=Hannahburgh, user_id=ee2e6d3f-ef10-4e9c-af76-fdc6c1594710>
Delete user_metris: UserMetrics <city=Lake Melissaside, user_id=bea23348-feac-444d-9b36-8333ba7aed13>
Delete user_metris: UserMetrics <city=Angelaberg, user_id=95f7dd94-ad5c-40a8-b7ad-438d5906c17f>
Delete user_metris: UserMetrics <city=Williamshaven, user_id=1abfb4f8-3286-4354-98d0-9ae6db2efe7a>
Delete user_metris: UserMetrics <city=East Timothy, user_id=879a163f-3be0-4fcc-b0c5-c84ff1800fe1>
Delete user_

In [22]:
# from cassandra.cqlengine import connection

# # This isn't a compaction, but it forces RAM data to disk so the background compaction process can see it.
# connection.execute(f"ALTER TABLE {keyspace_name}.user_metrics WITH gc_grace_seconds = 10")